In [1]:
import SimpleITK as sitk
import os
import pandas as pd
import json
import numpy as np

In [2]:
# data path
input_directory = 'C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng'
input_downsampled_directory = r'C:\Users\lydia\OneDrive\Work\DigitalAssets\_UPENN_fMOST\BIL_DATA\proj\u19zeng\fMOST_downsampled\brain'
output_directory = 'C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng'
output_downsampled_directory = r'C:\Users\lydia\OneDrive\Work\DigitalAssets\_UPENN_fMOST\BIL_DATA\proj\u19zeng\fMOST_downsampled\downsampled_converted'

In [3]:
# open combined and downsampled inventory files
combined_file = 'combined_inventory.csv'
downsample_file = 'downsampled_images_inventory.csv'

In [4]:
samples = pd.read_csv(os.path.join(input_directory,combined_file))
downsamples = pd.read_csv(os.path.join(input_directory,downsample_file))

samples.set_index('HUST Sample ID', inplace=True)
downsamples.set_index('Filename', inplace=True)

In [5]:
hlist = [191807,182709,182724,191812,192343,182725,191813,192337,192338,192339]
matched = [x in hlist for x in list(samples.index)]
samples = samples[matched]

In [6]:
samples

,Sample ID,Sample Type (CV),Species (CV),Species NCBI Taxonomy ID,Parent Specimen ID,Parent Specimen Type (CV),Subject ID,Age,Sex (CV),Genotype,...,Comments,Metadata Submission,AIBS LABTrack ID,Directories,TXT Files,JSON Files,Downsample Files,Red Downsample File,Green Downsample File,Number of Reconstructions
HUST Sample ID,,,,,,,,,,,,,,,,,,,,,
191812,mouseID_380470-191812,whole brain,mouse,NCBI:txid10090,NaN,NaN,380470,13w,female,Plxnd1-CreER/wt;Ai166(TIT2L-MORF-ICL-tTA2)/wt,...,NaN,2019Q4,380470,.;..;green;red,191812-metainfo.txt,191812.json,mouse191812green_xy32z8.tif;mouse191812red_xy3...,mouse191812red_xy32z8.tif,mouse191812green_xy32z8.tif,32.0
191813,mouseID_380471-191813,whole brain,mouse,NCBI:txid10090,NaN,NaN,380471,13w,female,Plxnd1-CreER/wt;Ai166(TIT2L-MORF-ICL-tTA2)/wt,...,NaN,2019Q4,380471,.;..;green;red,191813-metainfo.txt,191813.json,mouse191813green_xy32z8.tif;mouse191813red_xy3...,mouse191813red_xy32z8.tif,mouse191813green_xy32z8.tif,NaN
182724,mouseID_405426-182724,whole brain,mouse,NCBI:txid10090,NaN,NaN,405426,NaN,male,Fezf2-CreER/wt;Ai166(TIT2L-MORF-ICL-tTA2)/wt,...,NaN,2019Q3,405426,.;..;green;red,metainfo.txt,182724.json,mouse182724green_xy32z8.tif;mouse182724red_xy3...,mouse182724red_xy32z8.tif,mouse182724green_xy32z8.tif,22.0
182725,mouseID_405429-182725,whole brain,mouse,NCBI:txid10090,NaN,NaN,405429,NaN,male,Fezf2-CreER/wt;Ai166(TIT2L-MORF-ICL-tTA2)/wt,...,NaN,2019Q3,405429,.;..;green;red,metainfo.txt,182725.json,mouse182725green_xy32z8.tif;mouse182725red_xy3...,NaN,mouse182725green_xy32z8.tif,15.0
191807,mouseID_439168-191807,whole brain,mouse,NCBI:txid10090,NaN,NaN,439168,13w,female,Fezf2-CreER/wt;Ai166(TIT2L-MORF-ICL-tTA2)/wt,...,NaN,2019Q4,439168,.;..;green;red,191807-metainfo.txt,191807.json,mouse191807green_xy32z8.tif;mouse191807red_xy3...,mouse191807red_xy32z8.tif,mouse191807green_xy32z8.tif,13.0
192343,mouseID_454427-192343,whole brain,mouse,NCBI:txid10090,NaN,NaN,454427,18w,female,Plxnd1-CreER/wt;Ai166(TIT2L-MORF-ICL-tTA2)/wt,...,NaN,2021Q1,454427,.;..;green;red,192343-metainfo.txt;readme.txt,192343.json,mouse192343green_xy32z8.tif,NaN,mouse192343green_xy32z8.tif,NaN
192337,mouseID_452669-192337,whole brain,mouse,NCBI:txid10090,NaN,NaN,452669,13w,male,Nxph4-T2A-CreERT2/wt;Ai166(TIT2L-MORF-ICL-tTA2...,...,NaN,2021Q1,452669,.;..;green;red,192337-metainfo.txt;readme.txt,192337.json,mouse192337green_xy32z8.tif,NaN,mouse192337green_xy32z8.tif,NaN
192338,mouseID_452670-192338,whole brain,mouse,NCBI:txid10090,NaN,NaN,452670,14w,female,Nxph4-T2A-CreERT2/wt;Ai166(TIT2L-MORF-ICL-tTA2...,...,NaN,2021Q1,452670,.;..;green;red,192338-metainfo.txt;readme.txt,192338.json,mouse192338green_xy32z8.tif,NaN,mouse192338green_xy32z8.tif,NaN
192339,mouseID_452674-192339,whole brain,mouse,NCBI:txid10090,NaN,NaN,452674,13w,male,Nxph4-T2A-CreERT2/wt;Ai166(TIT2L-MORF-ICL-tTA2...,...,NaN,2021Q1,452674,.;..;green;red,192339-metainfo.txt;readme.txt,192339.json,mouse192339green_xy32z8.tif,NaN,mouse192339green_xy32z8.tif,NaN


In [7]:
oext = ".nii.gz"
compression = True


# Mapping from orientation tag to direction matrix and permutation to RAI
# To do - move this to a library
orientation = {}

orientation['RAI'] = {}
orientation['RAI']['direction'] = [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.1] # X:R->L, Y:A->P, Z:I->S
orientation['RAI']['permutation'] = [[0,False,1],[1,False,1],[2,False,0]]

orientation['SLA'] = {}
orientation['SLA']['direction'] = [0.0, -1.0, 0.0, 0.0, 0.0, 1.0, -1.0, 0.0, 0.0] # X:S->I, Y:L->R, Z:A->P
orientation['SLA']['permutation'] = [[1,True,0],[2,False,1],[0,True,1]]

orientation['ASL'] = {}
orientation['ASL']['direction'] = [ 0, 0, -1, 1, 0, 0, 0, -1, 0]  # X:A->P, Y:S->I, Z:L->R
orientation['ASL']['permutation'] = [[2,True,0],[0,False,1],[1,True,1]]

# represent the shifiting of the RAI origin posterior/left/inferior corner
sign = [-1.0,-1.0,1.0]
 
# mapping from rotation value in json to orientation
rotation = {}
rotation[0] = 'SLA'

for s, row in samples.iterrows() :
    
    print(s)
    
    dname = row['Sample ID']
    
    #
    # create output downsampled directory
    #
    odir = os.path.join( output_directory, dname, 'downsampled')
    if not os.path.exists( odir ) :
        os.makedirs( odir )
    
    #
    # open json metadata file
    #
    if pd.isna(row['JSON Files']) :
        continue
        
    jname = row['JSON Files'].split(';')[0]
    jpath = os.path.join( input_directory, dname, jname )
    
    with open(jpath,) as jf :
        jdata = json.load(jf)
        
    resolution = jdata['SampleInfo']['VoxelResolution ']
    resolution = np.multiply( resolution, 0.001 ) # convert to mm (many tools make this assumption)
    
    otag = rotation[jdata['SampleInfo']['Rotation']]
    direction = orientation[otag]['direction']
    
    #
    # convert downsample volume for each channel
    #
    ddf = downsamples
    ddf = ddf[ddf['HUST Sample ID'] == str(s)]
    ddf = ddf[ddf['Image Type'] == '.tif']

    for fname, frow in ddf.iterrows() :
        
        #if fname in exclude_list :
        #    continue

        c = frow['Channel']

        #fpath = os.path.join(input_directory, dname,fname)
        fpath = os.path.join(input_downsampled_directory, fname)
        print(" : " + fpath)

        meta = downsamples.loc[fname] 

        # compute spacing in mm
        downsample = [meta['XY Downsample'],meta['XY Downsample'],meta['Z Downsample']]
        spacing = np.multiply( downsample, resolution )

        # construct output file path
        #oname = "%d_%s_%0.4fx%0.4fx%0.4f_mm_%s%s" % (s,c,spacing[0],spacing[1],spacing[2],otag,oext)
        oname = "%d_%s_mm_%s%s" % (s,c,otag,oext)
        #opath = os.path.join(odir, oname)
        opath = os.path.join(output_downsampled_directory, oname)
        print("   : " + opath )

        # open volume
        volume = sitk.ReadImage( fpath )

        # set spacing
        volume.SetSpacing( spacing )

        # compute opposite corners
        opposite = [volume.GetSize()[d] - 1 for d in range(3)]
        opposite = volume.TransformIndexToPhysicalPoint(opposite)
        corners = [[0,p] for p in opposite]
        print("  :" + str(corners))

        # set origin
        origin = [ corners[d[0]][d[2]] for d in orientation[otag]['permutation'] ]
        origin = [ sign[d] * origin[d] for d in range(len(sign)) ]
        volume.SetOrigin( origin )
        print("  :" + str(origin))

        # set direction
        volume.SetDirection( direction )
        print("  :" + str(direction))

        # write output volume
        sitk.WriteImage( volume, opath, compression )
        



191812
 : C:\Users\lydia\OneDrive\Work\DigitalAssets\_UPENN_fMOST\BIL_DATA\proj\u19zeng\fMOST_downsampled\brain\mouse191812green_xy32z8.tif
   : C:\Users\lydia\OneDrive\Work\DigitalAssets\_UPENN_fMOST\BIL_DATA\proj\u19zeng\fMOST_downsampled\downsampled_converted\191812_green_mm_SLA.nii.gz
  :[[0, 6.5968], [0, 10.7632], [0, 11.648]]
  :[-0.0, -11.648, 6.5968]
  :[0.0, -1.0, 0.0, 0.0, 0.0, 1.0, -1.0, 0.0, 0.0]
 : C:\Users\lydia\OneDrive\Work\DigitalAssets\_UPENN_fMOST\BIL_DATA\proj\u19zeng\fMOST_downsampled\brain\mouse191812red_xy32z8.tif
   : C:\Users\lydia\OneDrive\Work\DigitalAssets\_UPENN_fMOST\BIL_DATA\proj\u19zeng\fMOST_downsampled\downsampled_converted\191812_red_mm_SLA.nii.gz
  :[[0, 6.5968], [0, 10.7632], [0, 11.648]]
  :[-0.0, -11.648, 6.5968]
  :[0.0, -1.0, 0.0, 0.0, 0.0, 1.0, -1.0, 0.0, 0.0]
191813
 : C:\Users\lydia\OneDrive\Work\DigitalAssets\_UPENN_fMOST\BIL_DATA\proj\u19zeng\fMOST_downsampled\brain\mouse191813green_xy32z8.tif
   : C:\Users\lydia\OneDrive\Work\DigitalAssets